In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
regensburg_pediatric_appendicitis = fetch_ucirepo(id=938) 
  
# data (as pandas dataframes) 
X = regensburg_pediatric_appendicitis.data.features 
y = regensburg_pediatric_appendicitis.data.targets 
  
# metadata 
print(regensburg_pediatric_appendicitis.metadata) 
  
# variable information 
print(regensburg_pediatric_appendicitis.variables) 

{'uci_id': 938, 'name': 'Regensburg Pediatric Appendicitis', 'repository_url': 'https://archive.ics.uci.edu/dataset/938/regensburg+pediatric+appendicitis', 'data_url': 'https://archive.ics.uci.edu/static/public/938/data.csv', 'abstract': 'This repository holds the data from a cohort of pediatric patients with suspected appendicitis admitted with abdominal pain to Children’s Hospital St. Hedwig in Regensburg, Germany, between 2016 and 2021. Each patient has (potentially multiple) ultrasound (US) images, aka views, tabular data comprising laboratory, physical examination, scoring results and ultrasonographic findings extracted manually by the experts, and three target variables, namely, diagnosis, management and severity.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Image'], 'num_instances': 782, 'num_features': 53, 'feature_types': ['Real', 'Categorical', 'Integer'], 'demographics': ['Age', 'Sex'], 'target_col': ['Management', 'Severity',

In [13]:
ultrasonic_images_path = 'H:/simnacma/CITs/regensburg_pediatric_data/preprocessed/constant_padding/deepfilled_cropped/'


{'ids': None,
 'features':        Age    BMI     Sex  Height  Weight  Length_of_Stay  Alvarado_Score  \
 0    12.68  16.90  female   148.0    37.0             3.0             4.0   
 1    14.10  31.90    male   147.0    69.5             2.0             5.0   
 2    14.14  23.30  female   163.0    62.0             4.0             5.0   
 3    16.37  20.60  female   165.0    56.0             3.0             7.0   
 4    11.08  16.90  female   163.0    45.0             3.0             5.0   
 ..     ...    ...     ...     ...     ...             ...             ...   
 777  12.41  25.25  female   166.5    70.0             4.0             8.0   
 778  17.09  20.43  female   158.0    51.0             6.0             5.0   
 779  14.99  19.91  female   152.0    46.0             4.0             5.0   
 780   7.20  14.30    male   129.3    23.9             5.0             9.0   
 781  11.51  18.17    male   146.5    39.0             4.0             2.0   
 
      Paedriatic_Appendicitis_Score 

In [4]:
X.loc[1]

Age                                   14.1
BMI                                   31.9
Sex                                   male
Height                               147.0
Weight                                69.5
Length_of_Stay                         2.0
Alvarado_Score                         5.0
Paedriatic_Appendicitis_Score          4.0
Appendix_on_US                          no
Appendix_Diameter                      NaN
Migratory_Pain                         yes
Lower_Right_Abd_Pain                   yes
Contralateral_Rebound_Tenderness       yes
Coughing_Pain                           no
Nausea                                  no
Loss_of_Appetite                       yes
Body_Temperature                      36.9
WBC_Count                              8.1
Neutrophil_Percentage                 64.8
Segmented_Neutrophils                  NaN
Neutrophilia                            no
RBC_Count                             5.26
Hemoglobin                            15.7
RDW        

In [9]:
import torch
import numpy as np
from PIL import Image
import open_clip

In [134]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')

In [135]:
model.eval()
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Context length: 77
Vocab size: 49408


In [136]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_to_rgb at 0x000001E917AA0180>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [138]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms

import medmnist
from medmnist import INFO, Evaluator

In [139]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v3.0.1 @ https://github.com/MedMNIST/MedMNIST/


In [140]:
data_flag = 'breastmnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 64
lr = 0.001

In [141]:
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [142]:
# load the data
train_dataset = DataClass(split='train', transform=preprocess, download=download, as_rgb=True)
test_dataset = DataClass(split='test', transform=preprocess, download=download, as_rgb=False)

pil_dataset = DataClass(split='train', download=download)

start_index = 0
end_index = 128  # Load first 128 samples, for example

# Create a Subset of the dataset containing only the specified indices
subset = torch.utils.data.Subset(train_dataset, range(start_index, end_index))

Using downloaded and verified file: C:\Users\Marco\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Marco\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Marco\.medmnist\breastmnist.npz


In [143]:
train_loader = data.DataLoader(dataset=subset, batch_size=BATCH_SIZE, shuffle=True)
pil_loader = data.DataLoader(dataset=pil_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [144]:
print(subset)

In [145]:
train_dataset.montage(length=1)

In [146]:
# List to store feature representations
features = []

# Iterate over the DataLoader to access batches of data
for images, _ in train_loader:
    # Apply transformations to the images
    # images = preprocess(images).to(device)

    # Obtain embeddings for the images
    with torch.no_grad():
        embeddings = model.encode_image(images).float()
        embeddings /= embeddings.norm(dim=-1, keepdim=True)

    # Append the embeddings to the features list
    features.append(embeddings.cpu().numpy())

In [151]:
print(features[0])
print(features[0].shape)
print(len(features))
print(features)
print(np.concatenate(features, axis=0).shape)

[[-0.0392431  -0.07876793 -0.02355219 ...  0.02742301 -0.00583899
  -0.02134407]
 [-0.04859295 -0.05071741  0.01970769 ...  0.02237301  0.01924399
  -0.02781691]
 [-0.04988016 -0.10676885 -0.01520137 ...  0.04452096 -0.00512853
  -0.03463099]
 ...
 [-0.04873145  0.01847215  0.01132574 ...  0.00362399  0.01225031
  -0.02893541]
 [-0.04174479 -0.0778585   0.01112214 ...  0.00762837  0.00323934
  -0.05025019]
 [-0.01384686  0.00228164 -0.01217282 ...  0.01448514 -0.01558171
  -0.02966568]]
(64, 512)
2
[array([[-0.0392431 , -0.07876793, -0.02355219, ...,  0.02742301,
        -0.00583899, -0.02134407],
       [-0.04859295, -0.05071741,  0.01970769, ...,  0.02237301,
         0.01924399, -0.02781691],
       [-0.04988016, -0.10676885, -0.01520137, ...,  0.04452096,
        -0.00512853, -0.03463099],
       ...,
       [-0.04873145,  0.01847215,  0.01132574, ...,  0.00362399,
         0.01225031, -0.02893541],
       [-0.04174479, -0.0778585 ,  0.01112214, ...,  0.00762837,
         0.0032393